# BGES Project : Questions

In [13]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

from sqlalchemy import create_engine
from sqlalchemy import text

import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
spark = SparkSession.builder \
    .config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
    .config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow") \
    .getOrCreate()

spark.conf.set("spark.sql.session.timeZone", "UTC")

## Vérification de l'ETL

In [15]:
DB_USER = 'postgres'
DB_PASSWORD = 'postgres'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'bges'

engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT 1"))
        print("✓ Successfully connected to the database")
except Exception as e:
    print(f"Error connecting to database: {e}")

✓ Successfully connected to the database


Example: List all tables

In [16]:
with engine.connect() as conn:
    tables = conn.execute(text("""
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
    """))
    print("\nTables in the database:")
    for table in tables:
        print(f"- {table[0]}")


Tables in the database:
- dim_sector
- dim_employee
- fact_employee_equipment
- fact_business_travel
- dim_equipment
- dim_date_time
- dim_mission_type
- dim_location
- dim_transport


In [17]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dim_transport"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from dim_transport:")
    print(df.head())


Rows from dim_transport:
   transport_id    transport_name emission_factor
0             1             plane          0.1994
1             2  public transport          0.1373
2             3             train          0.0106
3             4              taxi          0.1904


In [18]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dim_sector"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from dim_sector:")
    print(df)


Rows from dim_sector:
   sector_id         sector_name
0          1       data engineer
1          2  business executive
2          3           economist
3          4                 hrd
4          5   computer engineer


In [19]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dim_location"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from dim_location:")
    print(df.head())


Rows from dim_location:
   location_id         city    country
0            1       berlin  allemagne
1            2     shanghai      china
2            3  los angeles        usa
3            4       london    england
4            5        paris     france


In [20]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dim_date_time"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from dim_date_time:")
    print(df.head())


Rows from dim_date_time:
              date_id        date  day  month  year  hour  minute  second
0 2024-04-29 03:26:12  2024-04-29   29      4  2024     3      26      12
1 2024-04-29 03:10:27  2024-04-29   29      4  2024     3      10      27
2 2024-04-29 18:56:14  2024-04-29   29      4  2024    18      56      14
3 2024-04-29 14:55:58  2024-04-29   29      4  2024    14      55      58
4 2024-04-29 11:24:06  2024-04-29   29      4  2024    11      24       6


In [21]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dim_mission_type"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from dim_mission_type:")
    print(df)


Rows from dim_mission_type:
   mission_type_id mission_type_name
0                1          training
1                2           meeting
2                3        conference
3                4  business meeting
4                5       development


In [22]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dim_employee"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from dim_employee:")
    print(df.head())


Rows from dim_employee:
              employee_id last_name first_name  birth_date  birth_city  \
0  KeyPers_Berlin_1230000     Name0  FistName0  1993-11-04       osaka   
1  KeyPers_Berlin_1230001     Name1  FistName1  1932-11-22  wellington   
2  KeyPers_Berlin_1230002     Name2  FistName2  1990-08-12      sidney   
3  KeyPers_Berlin_1230003     Name3  FistName3  1965-05-26       rabat   
4  KeyPers_Berlin_1230004     Name4  FistName4  1959-01-18    shanghai   

  birth_country social_security_number phone_country_code   phone_number  \
0         japan            NS000000000                NaN  +336##0263188   
1   new zealand            NS000000001                NaN  +336##0401873   
2     australia            NS000000002                NaN  +336##0524126   
3         maroc            NS000000003                NaN  +336##0418484   
4         china            NS000000004                NaN  +336##0986317   

  address_street_number address_street_name address_complement postal_cod

In [23]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM dim_equipment"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from dim_equipment:")
    print(df.head())


Rows from dim_equipment:
  equipment_id      equipment_type              model co2_impact_kg
0            0  pc fixe sans ecran  modèle par défaut        350.00
1            1  pc fixe sans ecran     optiplex micro        174.00
2            2  pc fixe sans ecran     optiplex small        240.00
3            3  pc fixe sans ecran     optiplex tower        260.00
4            4  pc fixe sans ecran   wyse thin client         69.00


In [24]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM fact_business_travel"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from fact_business_travel:")
    print(df.head())


Rows from fact_business_travel:
          travel_id             employee_id  mission_type_id  \
0  BERLIN_202404290  KeyPers_Berlin_1233512                1   
1  BERLIN_202404291  KeyPers_Berlin_1233383                2   
2  BERLIN_202404292  KeyPers_Berlin_1231880                3   
3  BERLIN_202404293  KeyPers_Berlin_1230994                4   
4  BERLIN_202404294  KeyPers_Berlin_1233557                5   

   departure_location_id  destination_location_id  transport_id  \
0                      1                        7             1   
1                      1                        8             1   
2                     39                        9             1   
3                      1                        1             2   
4                      1                       10             1   

              date_id distance_km  is_round_trip  
0 2024-04-29 03:26:12     1635.08           True  
1 2024-04-29 03:10:27    13457.86           True  
2 2024-04-29 18:56:14    1

In [25]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM fact_employee_equipment"))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
    print("\nRows from fact_employee_equipment:")
    print(df.head())


Rows from fact_employee_equipment:
                      id_materiel equipment_id             employee_id  \
0  BERLIN_MATERIEL_INFO_202404290           41  KeyPers_Berlin_1234137   
1  LONDON_MATERIEL_INFO_202404290           49  KeyPers_London_1230774   
2  LONDON_MATERIEL_INFO_202404291           76  KeyPers_London_1230988   
3  LONDON_MATERIEL_INFO_202404292            0  KeyPers_London_1232692   
4  LONDON_MATERIEL_INFO_202404293           48  KeyPers_London_1233650   

     purchase_date_id  
0 2024-04-29 11:33:30  
1 2024-04-29 11:12:32  
2 2024-04-29 09:33:23  
3 2024-04-29 08:57:52  
4 2024-04-29 09:11:28  


## Questions

### Question 1 : Combien d'ingénieurs informaticiens travaillent sur le site de Paris ?

In [26]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT COUNT(*) as nb_computer_engineers
        FROM dim_employee e
        JOIN dim_sector s ON e.sector_id = s.sector_id
        WHERE s.sector_name = 'computer engineer'
        AND e.current_city = 'paris'
    """))
    count = result.fetchone()[0]
    print(f"Number of computer engineers in Paris: {count}")

Number of computer engineers in Paris: 1831


### Question 2 : Combien d'ingénieurs Data travaillent sur les sites de London ?

In [27]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT COUNT(*) as nb_data_engineers
        FROM dim_employee e
        JOIN dim_sector s ON e.sector_id = s.sector_id
        WHERE s.sector_name = 'data engineer'
        AND e.current_city = 'london'
    """))
    count = result.fetchone()[0]
    print(f"Number of data engineers in London: {count}")

Number of data engineers in London: 1568


### Question 3 : Combien de cadres travaillent dans l'organisation (tous sites compris) ?

In [28]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT COUNT(*) as nb_executives
        FROM dim_employee e
        JOIN dim_sector s ON e.sector_id = s.sector_id
        WHERE s.sector_name = 'business executive'
    """))
    count = result.fetchone()[0]
    print(f"Number of business executives across all sites: {count}")

Number of business executives across all sites: 3955


### Question 4 : Combien de PC portables ont été achetés par l'organisation entre mai et octobre 2024 ?

In [51]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT COUNT(*) as nb_laptops
        FROM fact_employee_equipment fee
        JOIN dim_equipment e ON fee.equipment_id = e.equipment_id
        JOIN dim_date_time dt ON fee.purchase_date_id = dt.date_id
        WHERE e.equipment_type = 'pc portable'
        AND dt.date BETWEEN '2024-05-01' AND '2024-10-31'
    """))
    count = result.fetchone()[0]
    print(f"Number of laptops purchased between May and October 2024: {count}")

Number of laptops purchased between May and October 2024: 2420


### Question 5 : Quelle a été l'impact carbone des PC portables entre mai et octobre 2024 ?

In [31]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT SUM(e.co2_impact_kg) as total_co2_impact
        FROM fact_employee_equipment fee
        JOIN dim_equipment e ON fee.equipment_id = e.equipment_id
        JOIN dim_date_time dt ON fee.purchase_date_id = dt.date_id
        WHERE e.equipment_type = 'pc portable'
        AND dt.date BETWEEN '2024-05-01' AND '2024-10-31'
    """))
    impact = result.fetchone()[0]
    print(f"Carbon impact of laptops between May and October 2024: {impact:.2f} kg CO2")

Carbon impact of laptops between May and October 2024: 647562.00 kg CO2


### Question 6 : Quelle a été l'impact carbone des PC fixes sans ecran achetés par les ingénieurs Data entre mai et septembre 2024 sur les sites de Paris et New-York ?

In [32]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT SUM(e.co2_impact_kg) as total_co2_impact
        FROM fact_employee_equipment fee
        JOIN dim_equipment e ON fee.equipment_id = e.equipment_id
        JOIN dim_date_time dt ON fee.purchase_date_id = dt.date_id
        JOIN dim_employee emp ON fee.employee_id = emp.employee_id
        JOIN dim_sector s ON emp.sector_id = s.sector_id
        WHERE e.equipment_type = 'pc fixe sans ecran'
        AND s.sector_name = 'data engineer'
        AND emp.current_city IN ('paris', 'new-york')
        AND dt.date BETWEEN '2024-05-01' AND '2024-09-30'
    """))
    impact = result.fetchone()[0]
    print(f"Carbon impact of desktop PCs purchased by data engineers in Paris and New York: {impact:.2f} kg CO2")

Carbon impact of desktop PCs purchased by data engineers in Paris and New York: 75670.00 kg CO2


### Question 7 : Quelle a été l'impact carbone des Ecrans achetés par les cadres entre mai et septembre 2024 sur tous les sites de l'organisation ?

In [34]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT SUM(e.co2_impact_kg) as total_co2_impact
        FROM fact_employee_equipment fee
        JOIN dim_equipment e ON fee.equipment_id = e.equipment_id
        JOIN dim_date_time dt ON fee.purchase_date_id = dt.date_id
        JOIN dim_employee emp ON fee.employee_id = emp.employee_id
        JOIN dim_sector s ON emp.sector_id = s.sector_id
        WHERE e.equipment_type = 'ecran'
        AND s.sector_name = 'business executive'
        AND dt.date BETWEEN '2024-05-01' AND '2024-09-30'
    """))
    impact = result.fetchone()[0]
    print(f"Carbon impact of monitors purchased by executives between May and September 2024: {impact:.2f} kg CO2")

Carbon impact of monitors purchased by executives between May and September 2024: 27290.00 kg CO2


### Question 8 : Quelle a été l'impact carbone des missions sur les sites Européens en juillet 2024 ?

In [45]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        JOIN dim_location dl ON bt.destination_location_id = dl.location_id
        WHERE dt.month = 7 AND dt.year = 2024
        AND dl.city IN ('paris', 'berlin', 'london')
    """))
    impact = result.fetchone()[0]
    print(f"Carbon impact of missions in European sites during July 2024: {impact:.2f} kg CO2")

Carbon impact of missions in European sites during July 2024: 334621.86 kg CO2


### Question 9 : Quels ont été les 5 jours les plus impactents concernant les missions en avion sur l'ensemble des sites de l'organisation ?

In [36]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT dt.date, SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        WHERE t.transport_name = 'plane'
        GROUP BY dt.date
        ORDER BY total_co2_impact DESC
        LIMIT 5
    """))
    print("Top 5 days with highest carbon impact from plane missions:")
    for i, row in enumerate(result, 1):
        print(f"{i}. {row[0]}: {row[1]:.2f} kg CO2")

Top 5 days with highest carbon impact from plane missions:
1. 2024-07-11: 532406.66 kg CO2
2. 2024-10-01: 501996.02 kg CO2
3. 2024-05-24: 495791.93 kg CO2
4. 2024-10-19: 494602.56 kg CO2
5. 2024-06-15: 485088.45 kg CO2


### Question 10 : Quel a été le secteur d'activité qui a eu le plus d'impact concernant les missions et le matériel informatique sur l'ensemble des sites de l'organisation ?

In [37]:
with engine.connect() as conn:
    result = conn.execute(text("""
        WITH mission_impact AS (
            SELECT s.sector_name,
                   SUM(bt.distance_km * t.emission_factor) as mission_co2_impact
            FROM fact_business_travel bt
            JOIN dim_employee e ON bt.employee_id = e.employee_id
            JOIN dim_sector s ON e.sector_id = s.sector_id
            JOIN dim_transport t ON bt.transport_id = t.transport_id
            GROUP BY s.sector_name
        ),
        equipment_impact AS (
            SELECT s.sector_name,
                   SUM(eq.co2_impact_kg) as equipment_co2_impact
            FROM fact_employee_equipment fee
            JOIN dim_employee e ON fee.employee_id = e.employee_id
            JOIN dim_sector s ON e.sector_id = s.sector_id
            JOIN dim_equipment eq ON fee.equipment_id = eq.equipment_id
            GROUP BY s.sector_name
        )
        SELECT m.sector_name,
               (COALESCE(m.mission_co2_impact, 0) + COALESCE(e.equipment_co2_impact, 0)) as total_co2_impact
        FROM mission_impact m
        FULL OUTER JOIN equipment_impact e ON m.sector_name = e.sector_name
        ORDER BY total_co2_impact DESC
        LIMIT 1
    """))
    sector, impact = result.fetchone()
    print(f"Sector with highest carbon impact: {sector} with {impact:.2f} kg CO2")

Sector with highest carbon impact: data engineer with 25917609.18 kg CO2


### Question 11 : Quel site a eu le plus d'impact concernant les missions et le matériel informatique sur l'ensemble des sites de l'organisation ?

In [38]:
with engine.connect() as conn:
    result = conn.execute(text("""
        WITH mission_impact AS (
            SELECT e.current_city,
                   SUM(bt.distance_km * t.emission_factor) as mission_co2_impact
            FROM fact_business_travel bt
            JOIN dim_employee e ON bt.employee_id = e.employee_id
            JOIN dim_transport t ON bt.transport_id = t.transport_id
            GROUP BY e.current_city
        ),
        equipment_impact AS (
            SELECT e.current_city,
                   SUM(eq.co2_impact_kg) as equipment_co2_impact
            FROM fact_employee_equipment fee
            JOIN dim_employee e ON fee.employee_id = e.employee_id
            JOIN dim_equipment eq ON fee.equipment_id = eq.equipment_id
            GROUP BY e.current_city
        )
        SELECT m.current_city,
               (COALESCE(m.mission_co2_impact, 0) + COALESCE(e.equipment_co2_impact, 0)) as total_co2_impact
        FROM mission_impact m
        FULL OUTER JOIN equipment_impact e ON m.current_city = e.current_city
        ORDER BY total_co2_impact DESC
        LIMIT 1
    """))
    city, impact = result.fetchone()
    print(f"Site with highest carbon impact: {city} with {impact:.2f} kg CO2")

Site with highest carbon impact: los angeles with 14429660.00 kg CO2


### Question 12 : Quel a été l'impact carbone des missions reliant chaque site (la ville de départ est un site de l'organisation et la ville d'arrivée également) durant le mois de mai 2024 ?

In [39]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT 
            SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        JOIN dim_location dl_dep ON bt.departure_location_id = dl_dep.location_id
        JOIN dim_location dl_dest ON bt.destination_location_id = dl_dest.location_id
        WHERE dt.year = 2024 
        AND dt.month = 5
        AND dl_dep.city IN ('berlin', 'paris', 'new-york', 'los angeles', 'shanghai', 'london')
        AND dl_dest.city IN ('berlin', 'paris', 'new-york', 'los angeles', 'shanghai', 'london')
    """))
    total_impact = result.scalar()
    
    if total_impact is None:
        print("Aucune mission entre sites n'a été enregistrée en mai 2024.")
    else:
        print(f"L'impact carbone total des missions entre sites en mai 2024 est de {total_impact:.2f} kg CO2")

L'impact carbone total des missions entre sites en mai 2024 est de 1269321.30 kg CO2


### Question 13 : Quel a été l'impact carbone des séminaires en juillet 2024 pour les employés de Los Angeles ?

In [40]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_employee e ON bt.employee_id = e.employee_id
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        JOIN dim_mission_type mt ON bt.mission_type_id = mt.mission_type_id
        WHERE e.current_city = 'los angeles'
        AND dt.month = 7 AND dt.year = 2024
        AND mt.mission_type_name = 'conference'
    """))
    impact = result.fetchone()[0]
    print(f"Carbon impact of seminars for Los Angeles employees in July 2024: {impact:.2f} kg CO2")

Carbon impact of seminars for Los Angeles employees in July 2024: 421958.71 kg CO2


### Question 14 : Quel a secteur d'activité a été le plus impactant pour les missions "conférences" entre mai et octobre 2024 ?

In [41]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT 
            s.sector_name,
            SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_employee e ON bt.employee_id = e.employee_id
        JOIN dim_sector s ON e.sector_id = s.sector_id
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        JOIN dim_mission_type mt ON bt.mission_type_id = mt.mission_type_id
        WHERE mt.mission_type_name = 'conference'
        AND dt.date BETWEEN '2024-05-01' AND '2024-10-31'
        GROUP BY s.sector_name
        ORDER BY total_co2_impact DESC
        LIMIT 1
    """))
    sector, impact = result.fetchone()
    print(f"Sector with highest carbon impact from conferences: {sector} with {impact:.2f} kg CO2")

Sector with highest carbon impact from conferences: data engineer with 4427602.98 kg CO2


### Question 15 : Quel a été l'âge moyen des employés qui sont partis en formations entre juillet et septembre 2024 ?

In [42]:
with engine.connect() as conn:
    result = conn.execute(text("""
        WITH employee_ages AS (
            SELECT DISTINCT e.employee_id,
                   DATE_PART('year', AGE(CAST('2024-09-30' AS DATE), e.birth_date)) as age
            FROM fact_business_travel bt
            JOIN dim_employee e ON bt.employee_id = e.employee_id
            JOIN dim_date_time dt ON bt.date_id = dt.date_id
            JOIN dim_mission_type mt ON bt.mission_type_id = mt.mission_type_id
            WHERE mt.mission_type_name = 'training'
            AND dt.date BETWEEN '2024-07-01' AND '2024-09-30'
        )
        SELECT AVG(age) as average_age
        FROM employee_ages
    """))
    avg_age = result.fetchone()[0]
    print(f"Average age of employees who went on training missions: {avg_age:.1f} years")

Average age of employees who went on training missions: 50.0 years


### Question 16 : Quelle destination a été la plus impactante (en cumul) entre mai et octobre 2024 ?

In [43]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT 
            dl.city as destination_city,
            dl.country as destination_country,
            SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_location dl ON bt.destination_location_id = dl.location_id
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        WHERE dt.date BETWEEN '2024-05-01' AND '2024-10-31'
        GROUP BY dl.city, dl.country
        ORDER BY total_co2_impact DESC
        LIMIT 1
    """))
    city, country, impact = result.fetchone()
    print(f"Most impactful destination: {city}, {country} with {impact:.2f} kg CO2")

Most impactful destination: melbourne, australia with 4156005.81 kg CO2


### Question 17 : Quelles ont été les trois catégories de missions les plus impactantes pour les cadres dans les sites Européens en juin 2024 ?

In [46]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT 
            mt.mission_type_name,
            SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_employee e ON bt.employee_id = e.employee_id
        JOIN dim_sector s ON e.sector_id = s.sector_id
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        JOIN dim_mission_type mt ON bt.mission_type_id = mt.mission_type_id
        JOIN dim_location dl ON bt.destination_location_id = dl.location_id
        WHERE s.sector_name = 'business executive'
        AND dl.city IN ('paris', 'berlin', 'london')
        AND dt.month = 6 AND dt.year = 2024
        GROUP BY mt.mission_type_name
        ORDER BY total_co2_impact DESC
        LIMIT 3
    """))
    print("Top 3 most impactful mission types for executives in European sites:")
    for i, row in enumerate(result, 1):
        print(f"{i}. {row[0]}: {row[1]:.2f} kg CO2")

Top 3 most impactful mission types for executives in European sites:
1. conference: 25132.61 kg CO2
2. development: 18278.05 kg CO2
3. business meeting: 15327.25 kg CO2


## 2. Questions à répondre avec des illustrations

### Question 18 : Quelles ont été les 5 missions les plus impactentes sur le site de Paris ?

In [47]:
with engine.connect() as conn:
    result = conn.execute(text("""
        WITH mission_impact AS (
            SELECT 
                fbt.travel_id,
                fbt.distance_km,
                dt.emission_factor,
                fbt.distance_km * dt.emission_factor as total_impact,
                dl_depart.city as departure_city,
                dl_dest.city as destination_city,
                dt.transport_name
            FROM fact_business_travel fbt
            JOIN dim_transport dt ON fbt.transport_id = dt.transport_id
            JOIN dim_location dl_depart ON fbt.departure_location_id = dl_depart.location_id
            JOIN dim_location dl_dest ON fbt.destination_location_id = dl_dest.location_id
            WHERE fbt.travel_id LIKE 'Paris_%'
        )
        SELECT 
            travel_id,
            departure_city,
            destination_city,
            transport_name,
            distance_km,
            emission_factor,
            ROUND(total_impact::numeric, 2) as total_impact_kg_co2
        FROM mission_impact
        ORDER BY total_impact DESC
        LIMIT 5
    """))
    df = pd.DataFrame(result.fetchall(), columns=['Travel ID', 'Departure', 'Destination', 'Transport', 'Distance (km)', 'Emission Factor', 'Total Impact (kg CO2)'])
    
    # Create a bar plot
    plt.figure(figsize=(12, 6))
    sns.barplot(data=df, x='Travel ID', y='Total Impact (kg CO2)', hue='Transport')
    plt.title('Top 5 Most Impactful Missions from Paris Site')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('question18.pdf')
    plt.close()

### Question 19 : Proposer une figure comparant l'impact carbone mensuel des missions en fonction du type de transport et sur chaque site.

In [48]:
with engine.connect() as conn:
    result = conn.execute(text("""
        SELECT 
            dl_dep.city as departure_city,
            t.transport_name,
            dt.month,
            SUM(bt.distance_km * t.emission_factor) as total_co2_impact
        FROM fact_business_travel bt
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        JOIN dim_location dl_dep ON bt.departure_location_id = dl_dep.location_id
        WHERE dt.year = 2024
        GROUP BY dl_dep.city, t.transport_name, dt.month
        ORDER BY dl_dep.city, dt.month
    """))
    df = pd.DataFrame(result.fetchall(), columns=['City', 'Transport', 'Month', 'CO2 Impact'])
    
    plt.figure(figsize=(15, 8))
    g = sns.FacetGrid(df, col='City', col_wrap=3, height=4)
    g.map_dataframe(sns.lineplot, x='Month', y='CO2 Impact', hue='Transport')
    g.add_legend()
    plt.suptitle('Monthly Carbon Impact by Transport Type and City', y=1.02)
    plt.tight_layout()
    plt.savefig('question19.pdf')
    plt.close()

<Figure size 1500x800 with 0 Axes>

### Question 20 : Proposer une figure illustrant l'impact carbone global mensuel de l'organisation.

In [49]:
with engine.connect() as conn:
    # Get mission impact
    result = conn.execute(text("""
        SELECT 
            dt.month,
            SUM(bt.distance_km * t.emission_factor) as mission_co2_impact
        FROM fact_business_travel bt
        JOIN dim_transport t ON bt.transport_id = t.transport_id
        JOIN dim_date_time dt ON bt.date_id = dt.date_id
        WHERE dt.year = 2024
        GROUP BY dt.month
        ORDER BY dt.month
    """))
    df_missions = pd.DataFrame(result.fetchall(), columns=['Month', 'Mission Impact'])
    
    # Get equipment impact
    result = conn.execute(text("""
        SELECT 
            dt.month,
            SUM(e.co2_impact_kg) as equipment_co2_impact
        FROM fact_employee_equipment fee
        JOIN dim_equipment e ON fee.equipment_id = e.equipment_id
        JOIN dim_date_time dt ON fee.purchase_date_id = dt.date_id
        WHERE dt.year = 2024
        GROUP BY dt.month
        ORDER BY dt.month
    """))
    df_equipment = pd.DataFrame(result.fetchall(), columns=['Month', 'Equipment Impact'])
    
    # Combine the data
    df = pd.merge(df_missions, df_equipment, on='Month', how='outer').fillna(0)
    df['Total Impact'] = df['Mission Impact'] + df['Equipment Impact']
    
    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(df['Month'], df['Mission Impact'], label='Missions', marker='o')
    plt.plot(df['Month'], df['Equipment Impact'], label='Equipment', marker='o')
    plt.plot(df['Month'], df['Total Impact'], label='Total', marker='o', linewidth=2)
    plt.title('Monthly Global Carbon Impact of the Organization')
    plt.xlabel('Month')
    plt.ylabel('CO2 Impact (kg)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig('question20.pdf')
    plt.close()